In [1]:
# calculate the SLL invariant for a given set of points

import numpy as np
import pythonknot.alexander_poly as ap

points = ap.read_xyz("traj_knot31_L300_close.txt")

In [2]:
print(points.shape)

(10000, 300, 3)


In [6]:
# define the function to calculate the SLL invariant
# $M_{ij}=(\dot{r}_i\times\dot{r}_j)\cdot\frac{\vec{r}_i-\vec{r}_j}{|\vec{r}_i-\vec{r}_j|^3}$
# r_i is the position of the i-th point
# \dot{r}_i is the tangent vector at the i-th point, use finite difference to calculate it
# $SLL=\frac1{8\pi}\sum_{i=4}^N\sum_{j=3}^{i-1}\sum_{k=2}^{j-1}\sum_{\ell=1}^{k-1}M_{ik}M_{j\ell}$
def compute_Mij(r_i,r_j,dr_i,dr_j):
    r_ij = r_i - r_j
    r_ij_norm = np.linalg.norm(r_ij)
    return np.dot(np.cross(dr_i,dr_j),r_ij)/r_ij_norm**3

def calculate_dr(points,chain_type='open'):
    n = points.shape[0]
    dr = np.zeros((n,3))
    for i in range(1,n-1):
        dr[i] = (points[i+1]-points[i-1])/2
    if chain_type == 'ring':
        dr[0] = (points[1]-points[-1])
        dr[-1] = (points[0]-points[-2])
    elif chain_type == 'open':
        dr[0] = (points[1]-points[0])
        dr[-1] = (points[-1]-points[-2])
    return dr

def SLL(points,chain_type='open'):
    """ input is a numpy array of shape (n,3) where n is the number of points"""
    n = points.shape[0]
    M = np.zeros((n,n))
    d_r = calculate_dr(points,chain_type)
    for i in range(n):
        for j in range(i+1,n):
            M[i,j] = compute_Mij(points[i],points[j],d_r[i],d_r[j])
            M[j,i] = M[i,j]
    
    SLL = 0
    for i in range(3,n):
        for j in range(2,i):
            for k in range(1,j):
                for l in range(k):
                    SLL += M[i,k]*M[j,l]
    return SLL/(8*np.pi)/6

In [10]:
points = ap.read_xyz("traj_knot31_L300_open.txt")
print(points.shape)
print(SLL(points[0],chain_type='ring'))

(1000, 300, 3)
0.9979596406177468
